<a href="https://colab.research.google.com/github/Dong2Yo/DATA3960_1252/blob/main/%5CLectures%5CData_Cleaning_Transfomration_CalgaryProperty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning & Transformation

What this notebook demonstrates:

* One defensible end-to-end cleaning strategy

* Business-rule–driven logic:

* Status-based sold price handling

* Community-based imputation

* Structural field enforcement

* Explicit trade-offs (not “fix everything”)

* Feature engineering for analytics & Power BI

# ---------------------------------------------
# STRATEGY 1: Handling Missing and Null Values
# ---------------------------------------------
# - Identify structurally required fields (e.g., Address, Listing ID)
# - Decide between row removal vs. imputation
# - Replace system placeholders (0, '', 'UNKNOWN') with NULL
# - Use group-based imputation (e.g., median price by Community)


# ---------------------------------------------
# STRATEGY 2: Validating Domain-Specific Formats
# ---------------------------------------------
# - Apply regex validation for Canadian postal codes (A1A 1A1)
# - Enforce numeric bounds (e.g., Yr Built <= current year)
# - Validate bathroom increments (0.5 steps)


# ---------------------------------------------
# STRATEGY 3: Identifying and Treating Outliers
# ---------------------------------------------
# - Use IQR or Z-score methods for prices and square footage
# - Flag impossible values (Frontage > 1000m)
# - Separate legitimate luxury properties from data errors


# ---------------------------------------------
# STRATEGY 4: Resolving Temporal Inconsistencies
# ---------------------------------------------
# - Ensure Sold Date >= Listing Date
# - Set Sold Price to NULL for non-sold statuses
# - Recalculate Days on Market fields where needed


# ---------------------------------------------
# STRATEGY 5: Standardizing Categorical Values
# ---------------------------------------------
# - Normalize casing and spelling (e.g., 'remax' vs 'RE/MAX')
# - Map legacy property styles to standardized categories
# - Maintain reference lookup tables


# ---------------------------------------------
# STRATEGY 6: Feature Engineering for Analytics
# ---------------------------------------------
# - Price per SqFt
# - Sold-to-List Price Ratio
# - Property Age at Sale
# - Market Segment Buckets

In [1]:
# ===========================================================
# 1. Imports & Load Data
# ===========================================================
import pandas as pd
import numpy as np
import re
from datetime import datetime

input_path = 'calgary_real_estate_synthetic.csv'
df = pd.read_csv(input_path)

pd.set_option('display.float_format', '{:,.2f}'.format)


In [2]:
# ===========================================================
# 2. Initial Audit
# ===========================================================

df.shape

df.isna().mean().sort_values(ascending=False)


,0
Sold Date,0.55
Address,0.00
Listing ID,0.00
Community,0.00
Postal Code,0.00
Style,0.00
Status,0.00
List Price,0.00
Prop Class,0.00
Sold Price,0.00


This notebook outlines a comprehensive data cleaning and transformation strategy for real estate data, focusing on business-rule–driven logic and explicit trade-offs. The process involves several key steps:

1.  **Data Loading & Initial Audit**: The process begins by loading the `calgary_real_estate_synthetic.csv` dataset into a pandas DataFrame. An initial audit is performed to understand the dataset's dimensions and the proportion of missing values across different columns.

2.  **Structural Fields Enforcement**: Rows with missing `Address` values, considered critical identifiers, are removed to ensure data integrity.

3.  **Postal Code Validation**: Canadian postal codes are validated against a regex pattern. Invalid postal codes are identified and replaced with 'UNKNOWN'.

4.  **Sold Price Logic**:
    *   Properties with a 'Status' other than 'S' (Sold) have their 'Sold Price' set to `NaN` as they are considered unsold.
    *   Short-form 'Sold Price' values (e.g., '350' meaning $350,000), identified as values less than 10,000, are corrected by multiplying them by 1,000.

5.  **Year Built Cleaning**:
    *   Invalid 'Yr Built' values (0 or future years) are replaced with `NaN`.
    *   Missing 'Yr Built' values for non-vacant lots ('Prop Class' != 'VLOT') are imputed using the median 'Yr Built' of their respective 'Community'.

6.  **Frontage Cleaning**:
    *   Zero and extreme outlier values (greater than 1000) in 'FrontageM' are replaced with `NaN`.
    *   Missing 'FrontageM' values are imputed using the mode of 'FrontageM' within their respective 'Community'.

7.  **Beds & Baths Validation**:
    *   'Baths' values are validated to ensure they are in 0.5 increments; otherwise, they are set to `NaN`.
    *   Negative 'Beds' values are identified and replaced with `NaN`.

8.  **Temporal Consistency**: 'Sold Date' values are checked against 'Listing Date'. If a 'Sold Date' is earlier than its corresponding 'Listing Date', it is considered invalid and set to `NaN`.

9.  **Feature Engineering**: Several new features are engineered for analytics and Power BI reporting:
    *   `List Price / SqFt`
    *   `Sold Price / SqFt`
    *   `Sold/List Ratio`
    *   `Property Age` (calculated as `current_year - Yr Built`)

10. **Export Clean Dataset**: Finally, the cleaned and transformed DataFrame is exported to a new CSV file named `calgary_real_estate_cleaned_solution.csv`, without the DataFrame index.

In [3]:
# ===========================================================
# 3. Structural Fields
# ===========================================================

# Address is a critical identifier
initial_rows = len(df)
df = df.dropna(subset=['Address'])
removed_rows = initial_rows - len(df)
removed_rows


18

In [4]:
# ===========================================================
# 4. Postal Code Validation (Canada)
# ===========================================================

postal_pattern = re.compile(r'^[A-Z][0-9][A-Z] ?[0-9][A-Z][0-9]$')

def valid_postal(code):
    if pd.isna(code):
        return False
    return bool(postal_pattern.match(str(code).upper()))

invalid_postal = ~df['Postal Code'].apply(valid_postal)
df.loc[invalid_postal, 'Postal Code'] = 'UNKNOWN'


In [5]:
# ===========================================================
# 5. Sold Price Logic
# ===========================================================

# Unsold properties should not have a sold price
not_sold = df['Status'] != 'S'
df.loc[not_sold, 'Sold Price'] = np.nan

# Correct short-form sold prices
short_form = (df['Sold Price'] < 10000) & (df['Sold Price'].notna())
df.loc[short_form, 'Sold Price'] *= 1000


In [6]:
# ===========================================================
# 6. Year Built Cleaning
# ===========================================================

current_year = datetime.now().year

# Replace invalid years with NULL
df.loc[df['Yr Built'] == 0, 'Yr Built'] = np.nan
df.loc[df['Yr Built'] > current_year, 'Yr Built'] = np.nan

# Impute using community median (exclude vacant lots)
mask_non_vlot = df['Prop Class'] != 'VLOT'
df.loc[mask_non_vlot, 'Yr Built'] = (
    df.loc[mask_non_vlot]
      .groupby('Community')['Yr Built']
      .transform(lambda x: x.fillna(x.median()))
)


In [7]:
# ===========================================================
# 7. Frontage Cleaning
# ===========================================================

# Replace zeros and extreme outliers
df.loc[df['FrontageM'] == 0, 'FrontageM'] = np.nan
df.loc[df['FrontageM'] > 1000, 'FrontageM'] = np.nan

# Impute by community mode
frontage_mode = df.groupby('Community')['FrontageM'].transform(
    lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
)
df['FrontageM'] = frontage_mode


In [8]:
# ===========================================================
# 8. Beds & Baths Validation
# ===========================================================

# Baths must be in 0.5 increments
df.loc[(df['Baths'] * 2) % 1 != 0, 'Baths'] = np.nan

# Beds cannot be negative
df.loc[df['Beds'] < 0, 'Beds'] = np.nan


In [9]:
# ===========================================================
# 9. Temporal Consistency
# ===========================================================

invalid_dates = (df['Sold Date'].notna()) & (df['Sold Date'] < df['Listing Date'])
df.loc[invalid_dates, 'Sold Date'] = np.nan


In [10]:
# ===========================================================
# 10. Feature Engineering
# ===========================================================

df['List Price / SqFt'] = df['List Price'] / df['SqFt']
df['Sold Price / SqFt'] = df['Sold Price'] / df['SqFt']
df['Sold/List Ratio'] = df['Sold Price'] / df['List Price']
df['Property Age'] = current_year - df['Yr Built']


In [11]:
# ===========================================================
# 11. Export Clean Dataset
# ===========================================================

output_path = 'calgary_real_estate_cleaned_solution.csv'
df.to_csv(output_path, index=False)

output_path


'calgary_real_estate_cleaned_solution.csv'